In [31]:
import pandas as pd
df_en = pd.read_csv(r'C:\Users\XE\Downloads\1mcorpus\corpus.en_ru.1m.en', encoding="utf-8", names=["seq_en"], sep='\r\n')
df_ru = pd.read_csv(r'C:\Users\XE\Downloads\1mcorpus\corpus.en_ru.1m.ru', encoding="utf-8", names=["seq_ru"], sep='\r\n')

C:\Users\XE\AppData\Local\Temp\ipykernel_1348\3478127182.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df_en = pd.read_csv(r'C:\Users\XE\Downloads\1mcorpus\corpus.en_ru.1m.en', encoding="utf-8", names=["seq_en"], sep='\r\n')
C:\Users\XE\AppData\Local\Temp\ipykernel_1348\3478127182.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df_ru = pd.read_csv(r'C:\Users\XE\Downloads\1mcorpus\corpus.en_ru.1m.ru', encoding="utf-8", names=["seq_ru"], sep='\r\n')


In [32]:
df_en = df_en.sample(100000, random_state=0)
df_ru = df_ru.sample(100000, random_state=0)

In [33]:
df_en.to_csv("../datasets/NMT_en")
df_en.to_csv("../datasets/NMT_ru")

In [52]:
df = df_en.join(df_ru)

In [61]:
df.to_csv("../datasets/NMT_en_ru")

In [1]:
import torch.nn as nn
import torch
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import sys
from torch.utils.data import TensorDataset, DataLoader
import re
from pymorphy3 import MorphAnalyzer
from nltk.corpus import stopwords
from razdel import tokenize
from nltk.tokenize import word_tokenize
from navec import Navec

In [2]:
import torchtext
import torchdata
from torchtext import transforms
from torchdata import datapipes

In [68]:
data_pipe_ru = datapipes.iter.IterableWrapper(["../datasets/NMT_ru"])
data_pipe_en = datapipes.iter.IterableWrapper(["../datasets/NMT_en"])

In [69]:
data_pipe_ru = datapipes.iter.FileOpener(data_pipe_ru, mode='r', encoding='utf-8')
data_pipe_en = datapipes.iter.FileOpener(data_pipe_en, mode='r', encoding='utf-8')

In [70]:
data_pipe_ru = data_pipe_ru.parse_csv(skip_lines=0)
data_pipe_en = data_pipe_en.parse_csv(skip_lines=0)

In [71]:
def tokenize_en(data: str) -> list[str]:
  data1 = re.sub("([.!?])", " ", str(data))
  data2 = re.sub("[^\s^\w^']+", " ", data1)
  data3 = word_tokenize(data2)
  return data3

In [72]:
def tokenize_ru(data: str) -> list[str]:
  data1 = re.sub("([.!?])", " ", str(data))
  data2 = re.sub("[^\s^\w^']+", " ", data1)
  data3 = tokenize(data2)
  return list(map(lambda x: x.text , data3))

In [73]:
def yield_tokens_en(data_iter: datapipes.iter.IterDataPipe):
  # итерируемся по набору данных
  for example in data_iter:
    yield tokenize_en(example)

In [74]:
def yield_tokens_ru(data_iter: datapipes.iter.IterDataPipe):
  # итерируемся по набору данных
  for example in data_iter:
    yield tokenize_ru(example)

In [75]:
from torchtext.vocab import build_vocab_from_iterator

In [76]:
vocab_en = build_vocab_from_iterator(
    yield_tokens_en(data_pipe_en),
    min_freq=1,
    specials= ['<pad>', '<sos>', '<eos>', '<unk>'],
    special_first=True
)
vocab_en.set_default_index(vocab_en['<unk>'])

In [77]:
vocab_ru = build_vocab_from_iterator(
    yield_tokens_ru(data_pipe_ru),
    min_freq=1,
    specials= ['<pad>', '<sos>', '<eos>', '<unk>'],
    special_first=True
)
vocab_ru.set_default_index(vocab_ru['<unk>'])

In [78]:
from torchtext import transforms

In [79]:
def vocab_transform(vocab: torchtext.vocab.Vocab) -> transforms.Sequential:
    text_tranform = transforms.Sequential(
        transforms.VocabTransform(vocab=vocab),
        transforms.AddToken(vocab['<sos>'], begin=True),
        transforms.AddToken(vocab['<eos>'], begin=False)
    )
    return text_tranform

In [80]:
def apply_vocab_transform_en(sample):
    return (
        vocab_transform(vocab_en)(tokenize_en(sample)),
    )

In [81]:
def apply_vocab_transform_ru(sample):
    return (
        vocab_transform(vocab_ru)(tokenize_ru(sample)),
    )

In [82]:
data_pipe_en = data_pipe_en.map(apply_vocab_transform_en)

In [83]:
data_pipe_ru = data_pipe_ru.map(apply_vocab_transform_en)

In [84]:
def apply_padding_en(pair):
    return transforms.ToTensor(vocab_en['<pad>'])(list(pair))

data_pipe_en = data_pipe_en.map(apply_padding_en)

In [85]:
def apply_padding_ru(pair):
    return transforms.ToTensor(vocab_ru['<pad>'])(list(pair))

data_pipe_ru = data_pipe_ru.map(apply_padding_ru)

In [92]:
ts_en = list(data_pipe_en)
ts_ry = list(data_pipe_ru)

In [94]:
ts[4]

tensor([[    1, 64168,     4,   127,    40,  4013,  1969,   125, 32465,    33,
         21800,   378,    10,  8444,    11,   130,     4,     2]])